Librerias a usar:
langchain langchain-community
langserve chromadb beautifulsoup4 requests fastapi uvicorn

Ingestion de los datos y creacion de la base de datos vectorial con Chromadb

In [13]:
#Imports
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain import hub

In [14]:
#Ingestion de datos
# Leer el archivo txt con los datos de pdf
def read_text_file(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return f.read()

#Path al archivo
file_path = "C:/Users/aldoa/Documents/promtiorChallenge/data.txt"

text = read_text_file(file_path)
print(text)

In November 2022, ChatGPT was released, causing a significant impact and challenging previously unquestionable principles: 
Creativity is an exclusively human trait.
The speed of these technological advancements has put unprecedented pressure on leaders to incorporate AI into their businesses.
In May 2023, Promtior was founded facing this context, where the key question is: how to approach a scenario of transversal disruption and maximize the opportunities it presents?
Through its technological and organizational consulting, Promtior offers a way to generate new business models, answering this question and bringing companies at the forefront of their sector.
Organizations that started their transition towards a Bionic future, with the help of Promtior: Paigo, Handy, SIEE, CAF, 5M Travell Group, Forestal Atlántico Sur.
At Promtior, we specialize in implementing cutting-edge generative solutions, with a focus on the implementation of the RAG (Retrieval Augmented Generation) architecture 

In [21]:
#Modelo
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model = "llama3",
    temperature = 0.8,
    num_predict = 256,
    base_url="https://llama-bpe9g6g9gpd2bmbs.eastus2-01.azurewebsites.net:11343"
)

In [16]:
embeddings = OllamaEmbeddings(model="llama3")

vector_store = InMemoryVectorStore(embeddings)

# Dividir el texto en fragmentos manejables
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.create_documents([text])
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

# Index chunks
_ = vector_store.add_documents(documents=all_splits)


Split blog post into 2 sub-documents.


In [17]:
from langchain_core.documents import Document
from typing_extensions import List, TypedDict


class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

In [18]:
prompt = hub.pull("rlm/rag-prompt")

c:\Users\aldoa\AppData\Local\Programs\Python\Python312\Lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [23]:
from langgraph.graph import START, StateGraph

# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [26]:
result = graph.invoke({"question": "What services does Promtior offer?"})

print(f'Answer: {result["answer"]}')

ConnectTimeout: [WinError 10060] Se produjo un error durante el intento de conexión ya que la parte conectada no respondió adecuadamente tras un periodo de tiempo, o bien se produjo un error en la conexión establecida ya que el host conectado no ha podido responder

### Deploy del modelo

In [12]:
from fastapi import FastAPI
from langserve import add_routes
import uvicorn

app = FastAPI()

# Exponer el chatbot como API
add_routes(app, graph, path="/chatbot")



RuntimeError: asyncio.run() cannot be called from a running event loop